This is used to enable interactive shell

In [9]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

The file and folder structure is given below:

Tensorflow Object Detection
|-----Detection TF2.ipynb --> For detection

|-----Trainer TF2.ipynb --> For training

|-----addons

|-----models ->> github TFOD master

|-----scripts

        |-----exporter_main_v2.py

        |-----generate_tfrecord.py

        |-----model_main_tf2.py

|-----workspace

        |-----training

                |-----annotations --> Contains labelmap TF record files

                |-----exported-models --> Exported model after training

                |-----images --> Images for train, test, eval

                |-----pre-trained-models  --> Each folder containing downloaded pre-trained models

                |-----under-training-models --> Each folder containing customized pipeline config for the pretrained models and its checkpoints

Imports


In [10]:
import os
# os.environ['TF_XLA_FLAGS'] = '--tf_xla_enable_xla_devices'
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
import pathlib

In [11]:
print("Number of GPU:{}".format(len(tf.config.list_physical_devices("GPU"))))
#Limit GPU Usage
for gpu in tf.config.experimental.list_physical_devices('GPU'):
    tf.config.experimental.set_memory_growth(gpu, True)

#Use this command for seeing which operation is given which device
#tf.debugging.set_log_device_placement(True)


Number of GPU:1


Path to images and annotations(labelmap.pbtxt, train.record, test.record)

In [13]:
PATH_TO_TRAIN_IMAGES = pathlib.Path.cwd()/"workspace"/"training"/"images"/"train"
PATH_TO_TEST_IMAGES =  pathlib.Path.cwd()/"workspace"/"training"/"images"/"test"
PATH_TO_EVAL_IMAGES =  pathlib.Path.cwd()/"workspace"/"training"/"images"/"eval"
PATH_TO_ANNOTATIONS = pathlib.Path.cwd()/"workspace"/"training"/"annotations"
PATH_TO_EXPORTED_MODEL_DIR = pathlib.Path.cwd()/"workspace"/"training"/"exported-models"
PATH_TO_DETECTED_IMAGES = pathlib.Path.cwd()/"workspace"/"training"/"images"/"detected_images"

print(PATH_TO_TRAIN_IMAGES)
print(PATH_TO_TEST_IMAGES)
print(PATH_TO_EVAL_IMAGES)
print(PATH_TO_ANNOTATIONS)
print(PATH_TO_EXPORTED_MODEL_DIR)
print(PATH_TO_DETECTED_IMAGES)

D:\Deep Learning\Tensorflow Object Detection\workspace\training\images\train
D:\Deep Learning\Tensorflow Object Detection\workspace\training\images\test
D:\Deep Learning\Tensorflow Object Detection\workspace\training\images\eval
D:\Deep Learning\Tensorflow Object Detection\workspace\training\annotations
D:\Deep Learning\Tensorflow Object Detection\workspace\training\exported-models
D:\Deep Learning\Tensorflow Object Detection\workspace\training\images\detected_images


Download LabelImg and store it in addons. Use it to label images. Also create a label_map.pbtxt

In [5]:
#To be written. File already exists

The below code is used to create tfrecord files for train and test directories

In [10]:
#Skipping File already exists.

# # Create train data:
!python scripts/generate_tfrecord.py -x $PATH_TO_TRAIN_IMAGES -l $PATH_TO_ANNOTATIONS/label_map.pbtxt -o $PATH_TO_ANNOTATIONS/train.record

# # Create test data:
!python scripts/generate_tfrecord.py -x $PATH_TO_TEST_IMAGES -l $PATH_TO_ANNOTATIONS/label_map.pbtxt -o $PATH_TO_ANNOTATIONS/test.record


usage: generate_tfrecord.py [-h] [-x XML_DIR] [-l LABELS_PATH]
                            [-o OUTPUT_PATH] [-i IMAGE_DIR] [-c CSV_PATH]
generate_tfrecord.py: error: unrecognized arguments: Learning\Tensorflow Object Detection\workspace\training\images\train Learning\Tensorflow Object Detection\workspace\training\annotations/label_map.pbtxt Learning\Tensorflow Object Detection\workspace\training\annotations/train.record
usage: generate_tfrecord.py [-h] [-x XML_DIR] [-l LABELS_PATH]
                            [-o OUTPUT_PATH] [-i IMAGE_DIR] [-c CSV_PATH]
generate_tfrecord.py: error: unrecognized arguments: Learning\Tensorflow Object Detection\workspace\training\images\test Learning\Tensorflow Object Detection\workspace\training\annotations/label_map.pbtxt Learning\Tensorflow Object Detection\workspace\training\annotations/test.record


Training

python scripts/model_main_tf2.py --model_dir="D:\Deep Learning\Tensorflow Object Detection\workspace\training\under-training-models\faster_rcnn" --pipeline_config_path="D:\Deep Learning\Tensorflow Object Detection\workspace\training\under-training-models\faster_rcnn\pipeline.config"

Export

python scripts\exporter_main_v2.py --input_type image_tensor --pipeline_config_path ./workspace/training/under-training-models/faster_rcnn/pipeline.config --trained_checkpoint_dir ./workspace/training/under-training-models/faster_rcnn/ --output_directory ./workspace/training/exported-models

Testing

In [18]:
def save_detected_image(image, name):
    try:
        PATH_TO_DETECTED_IMAGES.mkdir()
    except FileExistsError: print("Folder already exists")

    name = str(name)+".jpg"
    image = Image.fromarray(image)
    image.save(PATH_TO_DETECTED_IMAGES/name)
    print("Detected image saved as", name)



In [19]:
import time
from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

PATH_TO_CFG = pathlib.Path(PATH_TO_EXPORTED_MODEL_DIR, "10200", "pipeline.config")
PATH_TO_CKPT = pathlib.Path(PATH_TO_EXPORTED_MODEL_DIR, "10200", "checkpoint")

print('Loading model... ', end='')
start_time = time.time()

# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(PATH_TO_CFG)
model_config = configs['model']
detection_model = model_builder.build(model_config=model_config, is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(PATH_TO_CKPT, 'ckpt-0')).expect_partial()

@tf.function
def detect_fn(image):
    """Detect objects in image."""

    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)

    return detections

end_time = time.time()
elapsed_time = end_time - start_time
print('Done! Took {} seconds'.format(elapsed_time))


Loading model... 

Done! Took 0.2575511932373047 seconds


In [20]:
%matplotlib inline

import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')   # Suppress Matplotlib warnings

category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_ANNOTATIONS/"label_map.pbtxt", use_display_name=True)

def load_image_into_numpy_array(path):
    """Load an image from file into a numpy array.

    Puts image into numpy array to feed into tensorflow graph.
    Note that by convention we put it into a numpy array with shape
    (height, width, channels), where channels=3 for RGB.

    Args:
      path: the file path to the image

    Returns:
      uint8 numpy array with shape (img_height, img_width, 3)
    """
    return np.array(Image.open(path).convert("RGB"))


def run_inference(image_path):

    print('Running inference for {}... '.format(image_path), end='')

    image_np = load_image_into_numpy_array(image_path)

    # Things to try:
    # Flip horizontally
    # image_np = np.fliplr(image_np).copy()

    # Convert image to grayscale
    # image_np = np.tile(
    #     np.mean(image_np, 2, keepdims=True), (1, 1, 3)).astype(np.uint8)

    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)

    detections = detect_fn(input_tensor)

    # All outputs are batches tensors.
    # Convert to numpy arrays, and take index [0] to remove the batch dimension.
    # We're only interested in the first num_detections.
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    label_id_offset = 1
    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
            image_np_with_detections,
            detections['detection_boxes'],
            detections['detection_classes']+label_id_offset,
            detections['detection_scores'],
            category_index,
            use_normalized_coordinates=True,
            max_boxes_to_draw=200,
            min_score_thresh=.30,
            agnostic_mode=False)

    save_detected_image(image_np_with_detections, image_path.stem)


# sphinx_gallery_thumbnail_number = 2






In [21]:
for image_path in PATH_TO_TEST_IMAGES.glob("i*.jpg"):
    run_inference(image_path)

Running inference for D:\Deep Learning\Tensorflow Object Detection\workspace\training\images\test\i1.jpg... INFO:tensorflow:depth of additional conv before box predictor: 0
Folder already exists
Detected image saved as i1.jpg
Running inference for D:\Deep Learning\Tensorflow Object Detection\workspace\training\images\test\i10.jpg... Folder already exists
Detected image saved as i10.jpg
Running inference for D:\Deep Learning\Tensorflow Object Detection\workspace\training\images\test\i100.jpg... Folder already exists
Detected image saved as i100.jpg
Running inference for D:\Deep Learning\Tensorflow Object Detection\workspace\training\images\test\i101.jpg... Folder already exists
Detected image saved as i101.jpg
Running inference for D:\Deep Learning\Tensorflow Object Detection\workspace\training\images\test\i102.jpg... Folder already exists
Detected image saved as i102.jpg
Running inference for D:\Deep Learning\Tensorflow Object Detection\workspace\training\images\test\i103.jpg... Folder

Running inference for D:\Deep Learning\Tensorflow Object Detection\workspace\training\images\test\i143.jpg... Folder already exists
Detected image saved as i143.jpg
Running inference for D:\Deep Learning\Tensorflow Object Detection\workspace\training\images\test\i144.jpg... Folder already exists
Detected image saved as i144.jpg
Running inference for D:\Deep Learning\Tensorflow Object Detection\workspace\training\images\test\i145.jpg... Folder already exists
Detected image saved as i145.jpg
Running inference for D:\Deep Learning\Tensorflow Object Detection\workspace\training\images\test\i146.jpg... Folder already exists
Detected image saved as i146.jpg
Running inference for D:\Deep Learning\Tensorflow Object Detection\workspace\training\images\test\i147.jpg... Folder already exists
Detected image saved as i147.jpg
Running inference for D:\Deep Learning\Tensorflow Object Detection\workspace\training\images\test\i148.jpg... Folder already exists
Detected image saved as i148.jpg
Running in

Running inference for D:\Deep Learning\Tensorflow Object Detection\workspace\training\images\test\i189.jpg... Folder already exists
Detected image saved as i189.jpg
Running inference for D:\Deep Learning\Tensorflow Object Detection\workspace\training\images\test\i19.jpg... Folder already exists
Detected image saved as i19.jpg
Running inference for D:\Deep Learning\Tensorflow Object Detection\workspace\training\images\test\i190.jpg... Folder already exists
Detected image saved as i190.jpg
Running inference for D:\Deep Learning\Tensorflow Object Detection\workspace\training\images\test\i191.jpg... Folder already exists
Detected image saved as i191.jpg
Running inference for D:\Deep Learning\Tensorflow Object Detection\workspace\training\images\test\i192.jpg... Folder already exists
Detected image saved as i192.jpg
Running inference for D:\Deep Learning\Tensorflow Object Detection\workspace\training\images\test\i193.jpg... Folder already exists
Detected image saved as i193.jpg
Running infe

Running inference for D:\Deep Learning\Tensorflow Object Detection\workspace\training\images\test\i233.jpg... Folder already exists
Detected image saved as i233.jpg
Running inference for D:\Deep Learning\Tensorflow Object Detection\workspace\training\images\test\i234.jpg... Folder already exists
Detected image saved as i234.jpg
Running inference for D:\Deep Learning\Tensorflow Object Detection\workspace\training\images\test\i235.jpg... Folder already exists
Detected image saved as i235.jpg
Running inference for D:\Deep Learning\Tensorflow Object Detection\workspace\training\images\test\i236.jpg... Folder already exists
Detected image saved as i236.jpg
Running inference for D:\Deep Learning\Tensorflow Object Detection\workspace\training\images\test\i238.jpg... Folder already exists
Detected image saved as i238.jpg
Running inference for D:\Deep Learning\Tensorflow Object Detection\workspace\training\images\test\i239.jpg... Folder already exists
Detected image saved as i239.jpg
Running in

Running inference for D:\Deep Learning\Tensorflow Object Detection\workspace\training\images\test\i28.jpg... Folder already exists
Detected image saved as i28.jpg
Running inference for D:\Deep Learning\Tensorflow Object Detection\workspace\training\images\test\i280.jpg... Folder already exists
Detected image saved as i280.jpg
Running inference for D:\Deep Learning\Tensorflow Object Detection\workspace\training\images\test\i281.jpg... Folder already exists
Detected image saved as i281.jpg
Running inference for D:\Deep Learning\Tensorflow Object Detection\workspace\training\images\test\i282.jpg... Folder already exists
Detected image saved as i282.jpg
Running inference for D:\Deep Learning\Tensorflow Object Detection\workspace\training\images\test\i283.jpg... Folder already exists
Detected image saved as i283.jpg
Running inference for D:\Deep Learning\Tensorflow Object Detection\workspace\training\images\test\i284.jpg... Folder already exists
Detected image saved as i284.jpg
Running infe

Running inference for D:\Deep Learning\Tensorflow Object Detection\workspace\training\images\test\i54.jpg... Folder already exists
Detected image saved as i54.jpg
Running inference for D:\Deep Learning\Tensorflow Object Detection\workspace\training\images\test\i55.jpg... Folder already exists
Detected image saved as i55.jpg
Running inference for D:\Deep Learning\Tensorflow Object Detection\workspace\training\images\test\i56.jpg... Folder already exists
Detected image saved as i56.jpg
Running inference for D:\Deep Learning\Tensorflow Object Detection\workspace\training\images\test\i57.jpg... Folder already exists
Detected image saved as i57.jpg
Running inference for D:\Deep Learning\Tensorflow Object Detection\workspace\training\images\test\i58.jpg... Folder already exists
Detected image saved as i58.jpg
Running inference for D:\Deep Learning\Tensorflow Object Detection\workspace\training\images\test\i59.jpg... Folder already exists
Detected image saved as i59.jpg
Running inference for 

Run the below command for tensorboard. Make sure eval and train folder under the under-training-models/faster_rcnn exists

tensorboard --logdir=./workspace/training/under-training-models/faster_rcnn
